In [1]:
#makes import work
import pandas as pd
from scipy import stats
import numpy as np
import statistics
import copy
import scipy
import csv

# data = pd.read_excel("datasetfromHell.xlsx")
data = pd.read_excel("datasetfromHell_with_blanks.xlsx")
# data += 0.0000000000001

In [2]:
class DataColumn:
    def __init__(self, name, original):
        self.name = name
        self.original = original
        self.current = copy.deepcopy(original)
        self.path = []
        
    def info(self):
        print('name: {}\noriginal: {}\nCurrent: {}'.format(self.name, self.original, self.current))
   
    def shapiro(self):
            return stats.shapiro(self.current).pvalue
        
    def log_transform(self):
        self.current = list(np.log10(self.current))
        

In [3]:
class ComparisonResults:
    def __init__(self, name1, name2, path, testName, rho, pval, n):
        self.dataName1 = name1
        self.dataName2 = name2
        self.path = path
        self.testName = testName
        self.rho = rho
        self.pval = pval
        self.n = n
    
    def update_n(self, n):
        self.n = n
        
#     def info(self):
#         print('{} & {}\ntest: {}\nRho: {}\nP value: {}'.format(self.dataName1, self.dataName2, self.testName, self.rho, self.pval))
#         return
    
#     def to_csv(self):
#         return '{},{},{},{},{}'.format(self.dataName1, self.dataName2, self.testName, self.rho, self.pval)

In [4]:
def xor_less_than_p(a, b, p):
    a = float(a)
    b = float(b)
    if a < p and b >= p:
        return True
    elif b < p and a >= p:
        return True
    else:
        return False

In [5]:
def f_test(x, y):
    x = np.array(x)
    y = np.array(y)
    f = np.var(x, ddof=1)/np.var(y, ddof=1) #calculate F test statistic 
    dfn = x.size-1 #define degrees of freedom numerator 
    dfd = y.size-1 #define degrees of freedom denominator 
    p = 1-scipy.stats.f.cdf(f, dfn, dfd) #find p-value of F test statistic 
    return p

In [6]:
def test_decider(dataA, dataB):
    data1 = copy.deepcopy(dataA)
    data2 = copy.deepcopy(dataB)
    dataArray1 = data1.original
    dataArray2 = data2.original 
    
    # eliminate NaN
    df_both = pd.concat([dataArray1, dataArray2], axis=1)
    df_both = df_both.apply (pd.to_numeric, errors='coerce')
    df_both = df_both.dropna()
    df_both = df_both.reset_index(drop=True)
    df_both += 0.0000000000001
    
    data1 = DataColumn(data1.name, df_both.iloc[:,0])
    data2 = DataColumn(data2.name, df_both.iloc[:,1])

    n = data1.original.size
    p = 0.05000000000000000000000000000000000000000000000000
    path = []
    if xor_less_than_p(data1.shapiro(),data2.shapiro(),p):
        path.append('A')
        if data1.shapiro() > p:
            data1.log_transform()
        elif data2.shapiro() > p:
            data2.log_transform()
        if data1.shapiro() < p or data2.shapiro() < p:
            path.append('D')
            name1 = data1.name
            name2 = data2.name
            testName = 'Spearmans'
            rho, pval = stats.spearmanr(data1.original,data2.original)
            return(ComparisonResults(name1, name2, path, testName, rho, pval,n))
#             print('{} & {}:\nspearmans: {}'.format(data1.name, data2.name, path))
#             rho, pval = stats.spearmanr(data1.original,data2.original)
#             print('Rho: {}\nP val: {}\n'.format(rho, pval ))
            
        else:
            path.append('E')
            if f_test(data1.current, data2.current) > p:
                path.append('L')
                name1 = data1.name
                name2 = data2.name
                testName = 'Pearsons'
                rho, pval = stats.pearsonr(data1.current,data2.current)
                return(ComparisonResults(name1, name2, path, testName, rho, pval,n))
#                 print('{} & {}:\npearsons: {}'.format(data1.name, data2.name, path))
#                 rho, pval = stats.pearsonr(data1.current,data2.current)
#                 print('Rho: {}\nP val: {}\n'.format(rho, pval ))
            else:
                if data1.current == data1.original:
                    data1.log_transform()
                elif data2.current == data2.original:
                    data2.log_transform()
                if data1.shapiro() < p or data2.shapiro() < p:
                    path.append('G')
                    name1 = data1.name
                    name2 = data2.name
                    testName = 'Spearmans'
                    rho, pval = stats.spearmanr(data1.original,data2.original)
                    return(ComparisonResults(name1, name2, path, testName, rho, pval,n))

                else:
                    path.append('O')
                    name1 = data1.name
                    name2 = data2.name
                    testName = 'Pearsons'
                    rho, pval = stats.pearsonr(data1.current,data2.current)
                    return(ComparisonResults(name1, name2, path, testName, rho, pval,n))
    elif data1.shapiro() < p and data2.shapiro() < p:
        path.append('B')
        data1.log_transform()
        data2.log_transform()
        if data1.shapiro() < p or data2.shapiro() < p:
            path.append('M')
            name1 = data1.name
            name2 = data2.name
            testName = 'Spearmans'
            rho, pval = stats.spearmanr(data1.original,data2.original)
            return(ComparisonResults(name1, name2, path, testName, rho, pval,n))
        else:
            path.append('I')
            if f_test(data1.current, data2.current) < p:
                path.append('P')
                name1 = data1.name
                name2 = data2.name
                testName = 'Spearmans'
                rho, pval = stats.spearmanr(data1.original,data2.original)
                return(ComparisonResults(name1, name2, path, testName, rho, pval,n))
            else:
                path.append('Q')
                name1 = data1.name
                name2 = data2.name
                testName = 'Pearsons'
                rho, pval = stats.pearsonr(data1.current,data2.current)
                return(ComparisonResults(name1, name2, path, testName, rho, pval,n))
    else:
        path.append('C')
        if f_test(data1.current, data2.current) < p:
            path.append('F')
            data1.log_transform()
            data2.log_transform()
            if data1.shapiro() < p or data2.shapiro() < p:
                path.append('M')
                name1 = data1.name
                name2 = data2.name
                testName = 'Spearmans'
                rho, pval = stats.spearmanr(data1.original,data2.original)
                return(ComparisonResults(name1, name2, path, testName, rho, pval,n))
            else:
                path.append('I')
                if f_test(data1.current, data2.current) < p:
                    path.append('P')
                    name1 = data1.name
                    name2 = data2.name
                    testName = 'Spearmans'
                    rho, pval = stats.spearmanr(data1.original,data2.original)
                    return(ComparisonResults(name1, name2, path, testName, rho, pval,n))
                else:
                    path.append('Q')
                    name1 = data1.name
                    name2 = data2.name
                    testName = 'Pearsons'
                    rho, pval = stats.pearsonr(data1.current,data2.current)
                    return(ComparisonResults(name1, name2, path, testName, rho, pval,n))
        else:
            path.append('J')
            name1 = data1.name
            name2 = data2.name
            testName = 'Pearsons'
            rho, pval = stats.pearsonr(data1.current,data2.current)
            return(ComparisonResults(name1, name2, path, testName, rho, pval,n))
            

In [7]:
dataName = data.columns.values
dataArray = []

In [8]:
for i, name in enumerate(list(dataName)):
#     dataArray.append(DataColumn(name, data[name].tolist()))
    dataArray.append(DataColumn(name, data[name]))

In [9]:
results = []

In [10]:
for i in range(len(dataArray)):
    for j in range(len(dataArray)):
        results.append(test_decider(dataArray[i], dataArray[j]))

C:\Anaconda\lib\site-packages\scipy\stats\morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
C:\Anaconda\lib\site-packages\scipy\stats\stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
C:\Anaconda\lib\site-packages\scipy\stats\morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
C:\Anaconda\lib\site-packages\scipy\stats\stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [11]:
with open('Spearmans_Pearsons_Test_Results.csv', mode='w', newline='') as output_file:
    result_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    result_writer.writerow(['Attribute 1', 'Attribute 2', 'Test Run', 'n', 'Rho', 'P Value', 'Path'])
    for item in results:
        result_writer.writerow([item.dataName1, item.dataName2, item.testName, item.n, item.rho, item.pval, item.path])
    